In [1]:
import requests
import pandas as pd

In [2]:
df = pd.read_excel("./전처리_이진규.xlsx", sheet_name = 'Sheet1')

In [3]:
df2 = df[['발생일시', '관측소']].copy()

In [4]:
df2['발생일시']

0        2024-04-12 오전 10:30
1        2024-04-13 오전 08:47
2        2024-04-12 오전 09:00
3        2024-04-12 오전 10:30
4        2024-04-13 오전 11:30
                ...         
23717    2019-07-01 오전 07:10
23718    2019-07-01 오전 11:50
23719    2019-01-18 오후 03:30
23720    2019-07-01 오전 10:00
23721    2019-07-01 오전 08:50
Name: 발생일시, Length: 23722, dtype: object

In [5]:
tmp = df2['발생일시'].str.split('오')

In [6]:
tmp[0][0].replace(" ", "").replace("-", "")

'20240412'

In [7]:
def time_trans(x):
    x2 = x[1]
    plus = 0
    if x2[0] == '후':
        plus = 12
    x3 = int(x2[2:4]) + plus
    if x3 == 24:
        x3 = 12
    return "%02d" % x3

In [8]:
df2['Date'] = tmp.apply(lambda x : x[0].replace(" ", "").replace("-", ""))
df2['Hr'] = tmp.apply(lambda x : time_trans(x))

In [9]:
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

In [10]:
try:
    df_result = pd.read_excel("./기상정보_수집.xlsx")
except:
    df_result = pd.DataFrame()

In [11]:
df_result.head()

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te,사고명,발생일시,IDX
0,2019-01-18 15:00,1,232,천안,6.1,NaN,NaN,NaN,3.0,NaN,...,NaN,7.0,0.0,NaN,NaN,NaN,NaN,경기도 평택시 고덕면 삼성로 41 평택 EUV 신축공사,2019-01-18 오후 03:30,23719
1,2019-07-01 07:00,1,112,인천,20.2,NaN,NaN,NaN,0.4,NaN,...,19.0,20.8,NaN,21.9,22.1,22.5,22.6,경기도 부천시 소사본동 65-13 소사본동65-13 업무시설 신축공사_(주)순영종합건설,2019-07-01 오전 07:10,23717
2,2019-07-01 08:00,1,112,인천,20.9,NaN,NaN,NaN,0.9,NaN,...,19.0,23.0,NaN,21.9,22.1,22.4,22.5,경기도 부천시 심곡동 161-5 부천시 심곡동 161-5외 7필지 주상복합신축공사,2019-07-01 오전 08:50,23721
3,2019-07-01 10:00,1,112,인천,22.4,NaN,NaN,NaN,2.5,NaN,...,19.0,29.0,NaN,22.1,22.2,22.3,22.4,인천광역시 서구 가석로 257 (석남동) 서울도시철도7호선 석남연장 2공구 건설공사,2019-07-01 오전 10:00,23720
4,2019-07-01 10:00,1,232,천안,22.0,NaN,NaN,9.0,2.5,NaN,...,NaN,29.4,NaN,NaN,NaN,NaN,NaN,경기도 안성시 공도읍 공도로 116-24 안성공도 우방아이유쉘 공동주택 신축공사 중...,2019-07-01 오전 10:10,23716


In [12]:
is_error = False

for i in df2.index:
    try:
        try:
            if i in list(df_result['IDX'].astype('int')):
                continue
        except:
            pass
        if i % 100 == 0:
            print(i)
        HR = df2.loc[i, 'Hr']
        Date = df2.loc[i, 'Date']
        loc = df2.loc[i, '관측소']
        params = {'serviceKey' : 'GOcFuU1jXSl0WxsxO2SWIC10mJFK2PfS/YxIy++PbRAqXkBL0xVJ8JK7jfy266eavs7bE5BDTNM1n6Tb2dtJQg==',
    'pageNo' : '1', 'numOfRows' : '999', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'HR', 'startDt' : Date, 'startHh' : HR, 'endDt' : Date, 'endHh' : HR, 'stnIds' : loc }
        response = requests.get(url, params=params)
        tmp = eval(response.text)
        tmp_df = pd.DataFrame(tmp['response']['body']['items']['item'], index = [i])
        tmp_df.loc[i, '사고명'] = df.loc[i, '사고명']
        tmp_df.loc[i, '발생일시'] = df.loc[i, '발생일시']
        tmp_df.loc[i, 'IDX'] = i
        df_result = pd.concat([df_result, tmp_df], axis = 0)
        is_error = False
    except:
        print(f"{i}번째 인덱스에서 오류 발생")
        if is_error:
            print(f"{i}번째 인덱스에서 오류 연속 발생으로 종료")
            break
        is_error = True

0


In [13]:
df_result.to_excel("./기상정보_수집.xlsx", index = False)